<a href="https://colab.research.google.com/github/Darian-Lee-YTKA/RNN-Russian-Translator/blob/main/simple_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 825.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8f03b6e484dbe4bcf1bff75beeef671e224a7d45ac954b83aebef01b4466f811
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import pandas as pd
import pymorphy2
import random
import nltk
import re
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open('/content/rus.txt', 'r') as file:
    content = file.readlines()

def remove_punct(line):
    line = line.replace("n't", " not") # so conjuctions can be tokenized more easily
    line = line.replace("'m", " am")
    line = line.replace("let's", "let us")
    line = line.replace("'re", " are")
    line = line.replace("Tom's", "Toms") # so possestive doesn't become "is"
    line = line.replace("'s", " is")
    line = line.replace("'ve", " have")
    line = line.replace("'ll", " will")
    line = line.replace("won't", " will not")
    line = line.replace("'", " ")

    # I am going to keep the question make in since it effects word order
    punct_except_question_mark = ''.join(char for char in string.punctuation if char != '?')
    punct_regex = re.compile('[%s]' % re.escape(punct_except_question_mark))
    line = punct_regex.sub('', line)
    return line





russian_list = []
english_list = []

# the lines are grouped by complexity.


for line in content[:25000]: #only do the first 25,000 to keep the model a more managable size

# NOTE: this size will likely shrink even more once we delete translations that become duplicates once the Russian is lematized

    line = re.sub(r'\tCC-BY.*', '', line) #get rid of everything that is not data
    line = re.sub(r'\t', 'β', line) #make the lines easier to split. I was getting errors doing line.split based on tab,
    # so I replaced the tabs with beta

    line = remove_punct(line)

    eng, rus = line.split('β')

    rus = rus.strip().lower()  #to get rid of /n at end and make it all the same case

    eng = eng.strip().lower()

    russian_list.append(rus+' ')  #this will come in handy later when we add 'p' to tag past tense

    english_list.append(eng)



data = pd.DataFrame({'rus': russian_list, 'eng': english_list})
data = data.sample(frac=1, random_state=12) #shuffle it
russian_list = data['rus'].tolist()
english_list = data['eng'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data.shape

(25000, 2)

In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm

2024-01-02 20:23:14.860642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 20:23:14.860736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 20:23:14.862310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 20:23:16.403077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 774.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.2 MB/s eta 0:00:00

In [ ]:
for i in range(len(russian_list)): #for tagging past tense
# in russian, past tense is super easy to detect for most words because it will have one of these endings
    russian_list[i] = re.sub(r'л(?=\s|$)', 'л p', russian_list[i])
    russian_list[i] = re.sub(r'ла(?=\s|$)', 'ла p', russian_list[i])
    russian_list[i] = re.sub(r'лся(?=\s|$)', 'лся p', russian_list[i])
    russian_list[i] = re.sub(r'лась(?=\s|$)', 'лась p', russian_list[i])
    russian_list[i] = re.sub(r'ло(?=\s|$)', 'ло p', russian_list[i])
    russian_list[i] = re.sub(r'лось(?=\s|$)', 'лось p', russian_list[i])
    russian_list[i] = re.sub(r'ли(?=\s|$)', 'ли p', russian_list[i])
    russian_list[i] = re.sub(r'лись(?=\s|$)', 'лись p', russian_list[i])
    russian_list[i] = re.sub(r'\s+$', '', russian_list[i], flags=re.MULTILINE)


In [ ]:
russian_list

['я очень стара',
 'я зажатый',
 'не приходите сюда',
 'оно близко',
 'я обожаю тебя',
 'заедьте за ним',
 'он умеет читать',
 'том посмотрел p',
 'спрячьтесь здесь',
 'тебе том нравится?',
 'кажется ты хороший',
 'мыслите позитивно',
 'звоните в любое время',
 'ты пьян?',
 'теперь припоминаю',
 'наелись?',
 'подожди здесь',
 'не облажайся',
 'я парень занятой',
 'ты это отрицаешь?',
 'мне понравилось p',
 'теперь проваливай',
 'давайте быстрей',
 'я скучаю по тому',
 'можно мне с вами?',
 'я голосую против',
 'мне пришлось p подождать',
 'не беспокой меня',
 'это нам решать',
 'том облажался p',
 'сейчас я бодрствую',
 'у тебя бледный вид',
 'его там нет',
 'дома есть ктонибудь?',
 'не переусердствуй',
 'у меня грипп',
 'попробуй сделай это',
 'будь благоразумен',
 'сахар сладкий',
 'она ленивая',
 'я сожалею об этом',
 'я не силён',
 'будь очень осторожна',
 'я ещё приду',
 'я раньше курил p',
 'том был p ранен?',
 'мы подали p на них в суд',
 'это безумие',
 'они были p мертвы',
 'э

In [ ]:
import spacy

russian_lemmas = []

nlp = spacy.load("ru_core_news_sm")

counter = 0
for line in russian_list:
    counter+= 1
    print(counter) # so I can get an estimate for how much longer it will take by watching the numbers increase
    doc = nlp(line)
    lemmas = [token.lemma_ if token.text != 'p' else 'p' for token in doc] #so that the past tense remains tagged
    russian_lemmas.append(lemmas)


print(russian_lemmas)

In [ ]:
!pip install tensorflow

In [ ]:
# this will introduce duplicates but we will delete them later
from tensorflow.keras.preprocessing.text import Tokenizer

rus_tokenizer = Tokenizer(oov_token="<OOV>") #for handeling unknown tokens
eng_tokenizer = Tokenizer(oov_token="<OOV>")


rus_tokenizer.fit_on_texts(russian_lemmas)

rus_sequences = rus_tokenizer.texts_to_sequences(russian_lemmas)

rus_vocab_size = len(rus_tokenizer.word_index) + 1




eng_tokenizer.fit_on_texts(english_list)

eng_sequences = eng_tokenizer.texts_to_sequences(english_list)

eng_vocab_size = len(eng_tokenizer.word_index) + 1

final_data = pd.DataFrame({'rus': rus_sequences, 'eng': eng_sequences})


# as promised, we are deleting the dups
final_data = final_data.drop_duplicates(subset='rus', keep='first')


# so that we can do this in a seperate runtime to preserve RAM on google collab
import json

with open('/content/rus_tokenizer.json', 'w') as file:
    file.write(rus_tokenizer.to_json())

with open('/content/eng_tokenizer.json', 'w') as file:
    file.write(eng_tokenizer.to_json())

In [ ]:

final_data['rus_lens'] = final_data['rus'].apply(lambda x: len(x))

max_seq_rus = max(final_data['rus_lens'])
print(max_seq_rus)

final_data['eng_lens'] = final_data['eng'].apply(lambda x: len(x))

max_seq_eng = max(final_data['eng_lens'])
print(max_seq_rus)


10
10


<ipython-input-12-4e0660cb60be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['rus_lens'] = final_data['rus'].apply(lambda x: len(x))
<ipython-input-12-4e0660cb60be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['eng_lens'] = final_data['eng'].apply(lambda x: len(x))


In [ ]:
final_data['padded_rus'] = final_data['rus'].apply(lambda x: pad_sequences([x], maxlen=max_seq_rus, padding='post', truncating='post')[0])
final_data['padded_eng'] = final_data['eng'].apply(lambda x: pad_sequences([x], maxlen=max_seq_eng, padding='post', truncating='post')[0])



In [ ]:
# one hot encode the output:

from tensorflow.keras.utils import to_categorical

y = final_data['padded_eng'].apply(lambda x: to_categorical(x, num_classes=eng_vocab_size))



In [ ]:
import numpy as np
y_array = np.array(y.tolist())
print(y_array.shape)
y_reshaped = y_array.reshape(y_array.shape[0], y_array.shape[1], -1)

(19851, 5, 3212)


In [ ]:
y_array.shape

(19851, 5, 3212)

In [ ]:
from sklearn.model_selection import train_test_split

X = final_data['padded_rus']
y = y_reshaped

for x in y[:10]:
  print(len(x[0]))

temp_X, test_X, temp_y, test_y = train_test_split(X, y, test_size=0.1, random_state=24)


train_X, val_X, train_y, val_y = train_test_split(temp_X, temp_y, test_size=0.1, random_state=24)

3212
3212
3212
3212
3212
3212
3212
3212
3212
3212


In [ ]:
import numpy as np
train_X = np.array(train_X.values.tolist())

train_X = train_X.astype(np.float32)

test_X = np.array(test_X.values.tolist())

test_X = test_X.astype(np.float32)

val_X = np.array(val_X.values.tolist())

val_X = val_X.astype(np.float32)



In [ ]:
train_X.shape


(16078, 10)

In [ ]:
train_y.shape

(16078, 5, 3212)

In [ ]:
train_X

array([[  2.,  50.,   3., ...,   0.,   0.,   0.],
       [987.,   0.,   0., ...,   0.,   0.,   0.],
       [403.,  18., 179., ...,   0.,   0.,   0.],
       ...,
       [ 64.,  39.,   3., ...,   0.,   0.,   0.],
       [195.,  16.,   5., ...,   0.,   0.,   0.],
       [ 16., 116.,  29., ...,   0.,   0.,   0.]], dtype=float32)

In [ ]:
train_y

array([[[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

  

In [ ]:
# my RAM keeps running out, so this will buy me some time.
# I will restart my runtime and pick up from here
np.save('/content/train_X.npy', train_X)
np.save('/content/train_y.npy', train_y)
np.save('/content/test_X.npy', test_X)
np.save('/content/test_y.npy', test_y)
np.save('/content/val_X.npy', val_X)
np.save('/content/val_y.npy', val_y)

In [ ]:
import numpy as np

train_X = np.load('/content/train_X.npy')
train_y = np.load('/content/train_y.npy')
test_X = np.load('/content/test_X.npy')
test_y = np.load('/content/test_y.npy')
val_X = np.load('/content/val_X.npy')
val_y = np.load('/content/val_y.npy')

In [ ]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json


with open('/content/rus_tokenizer.json', 'r') as file:
    rus_tokenizer = tokenizer_from_json(file.read())

with open('/content/eng_tokenizer.json', 'r') as file:
    eng_tokenizer = tokenizer_from_json(file.read())

eng_vocab_size = len(eng_tokenizer.word_index) + 1
rus_vocab_size = len(rus_tokenizer.word_index) + 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.utils import plot_model

def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    return model



units = [128, 256]







In [ ]:
print(rus_vocab_size)
print(eng_vocab_size)

5660
3212


In [ ]:
from keras.callbacks import ModelCheckpoint
best_model = None
best_val_loss = float('inf')
for unit in units:
  model = create_model(rus_vocab_size, eng_vocab_size, 10, 5, unit)
  print(model.summary())
  filename = f'/content/best_model_{unit}.h5'
  checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  model.fit(train_X, train_y, epochs=30, batch_size=100, validation_data=(val_X, val_y), callbacks=[checkpoint], verbose=1)
  eval_loss = model.evaluate(test_X, test_y, verbose=1)

  print(f"testing loss for unit {unit}: {eval_loss}")

  if eval_loss < best_val_loss:
      best_val_loss = eval_loss
      best_model = model


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 128)           724480    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 repeat_vector_1 (RepeatVec  (None, 5, 128)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 5, 128)            131584    
                                                                 
 time_distributed_1 (TimeDi  (None, 5, 3212)           414348    
 stributed)                                                      
                                                                 
Total params: 1401996 (5.35 MB)
Trainable params: 1401

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


161/161 [==============================] - ETA: 0s - loss: 4.0593
Epoch 2: val_loss improved from 4.21917 to 3.97934, saving model to /content/best_model_128.h5
161/161 [==============================] - 35s 219ms/step - loss: 4.0593 - val_loss: 3.9793
Epoch 3/30
161/161 [==============================] - ETA: 0s - loss: 3.8375
Epoch 3: val_loss improved from 3.97934 to 3.80126, saving model to /content/best_model_128.h5
161/161 [==============================] - 28s 173ms/step - loss: 3.8375 - val_loss: 3.8013
Epoch 4/30
161/161 [==============================] - ETA: 0s - loss: 3.6910
Epoch 4: val_loss improved from 3.80126 to 3.70300, saving model to /content/best_model_128.h5
161/161 [==============================] - 28s 176ms/step - loss: 3.6910 - val_loss: 3.7030
Epoch 5/30
161/161 [==============================] - ETA: 0s - loss: 3.5932
Epoch 5: val_loss improved from 3.70300 to 3.61697, saving model to /content/best_model_128.h5
161/161 [==============================] - 32s 

In [ ]:
# because we got the best score with the larger model of the 2, I will try one more training, this time with 400 units. Anything over this would probably crash my RAM
from keras.callbacks import ModelCheckpoint
big_model = create_model(rus_vocab_size, eng_vocab_size, 10, 5, 400)
print(big_model.summary())
filename = f'/content/best_model_unit_400.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
big_model.fit(train_X, train_y, epochs=30, batch_size=100, validation_data=(val_X, val_y), callbacks=[checkpoint], verbose=1)




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 400)           2264000   
                                                                 
 lstm_2 (LSTM)               (None, 400)               1281600   
                                                                 
 repeat_vector_1 (RepeatVec  (None, 5, 400)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 5, 400)            1281600   
                                                                 
 time_distributed_1 (TimeDi  (None, 5, 3212)           1288012   
 stributed)                                                      
                                                                 
Total params: 6115212 (23.33 MB)
Trainable params: 611

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


161/161 [==============================] - 98s 564ms/step - loss: 4.1780 - val_loss: 3.4602
Epoch 2/30
161/161 [==============================] - ETA: 0s - loss: 3.2974
Epoch 2: val_loss improved from 3.46015 to 3.20060, saving model to /content/best_model_unit_400.h5
161/161 [==============================] - 87s 542ms/step - loss: 3.2974 - val_loss: 3.2006
Epoch 3/30
161/161 [==============================] - ETA: 0s - loss: 2.9519
Epoch 3: val_loss improved from 3.20060 to 2.86406, saving model to /content/best_model_unit_400.h5
161/161 [==============================] - 86s 534ms/step - loss: 2.9519 - val_loss: 2.8641
Epoch 4/30
161/161 [==============================] - ETA: 0s - loss: 2.6107
Epoch 4: val_loss improved from 2.86406 to 2.62482, saving model to /content/best_model_unit_400.h5
161/161 [==============================] - 86s 536ms/step - loss: 2.6107 - val_loss: 2.6248
Epoch 5/30
161/161 [==============================] - ETA: 0s - loss: 2.3364
Epoch 5: val_loss improv

In [ ]:
from tensorflow.keras.models import load_model

model_path = '/content/best_model_unit_400.h5'
big_model = load_model(model_path)

In [ ]:
big_model.evaluate(test_X, test_y, verbose=1)

63/63 [==============================] - 4s 60ms/step - loss: 1.5248


1.5248371362686157

the best model seems to be the one with 400 units

In [ ]:
translation = big_model.predict(test_X, verbose=1)
print(translation)

63/63 [==============================] - 11s 115ms/step
[[[4.30398011e-08 2.46817584e-12 9.94995415e-01 ... 5.96067642e-19
   1.66117185e-12 1.04383059e-19]
  [4.04719998e-07 2.07976880e-09 2.97707098e-04 ... 2.41531296e-13
   1.60793001e-09 1.03913080e-12]
  [1.49138284e-08 7.07315526e-11 4.86425888e-05 ... 2.20504343e-14
   5.29568958e-11 2.50583889e-14]
  [3.94827127e-03 7.95358162e-08 2.04193304e-04 ... 2.11184672e-12
   9.50405123e-08 2.14403287e-11]
  [9.99080777e-01 6.99312136e-11 1.09625137e-06 ... 5.84129643e-16
   9.42962444e-11 9.92301385e-15]]

 [[1.92966194e-08 1.16507791e-12 9.98167038e-01 ... 4.48111461e-20
   7.69835964e-13 5.26046157e-20]
  [6.64056259e-08 5.68297735e-11 2.37636996e-05 ... 1.62281401e-16
   3.97515215e-11 2.26176982e-15]
  [8.79313757e-06 2.80010326e-09 8.57503247e-03 ... 1.56708549e-13
   1.90689486e-09 1.05356236e-11]
  [1.89315025e-02 4.59230414e-08 1.90693231e-06 ... 4.99096814e-12
   4.22111768e-08 1.13668308e-09]
  [9.99911606e-01 2.86389901e-12 

In [ ]:
index_to_word = {index: word for word, index in eng_tokenizer.word_index.items()}


In [ ]:
import pandas as pd
translations = []
for sentence in translation:
  output = ''
  for word in sentence:
    token = np.argmax(word)
    if token == 0:
      continue
    else:
      eng_word = index_to_word[token]
    output+=eng_word
  translations.append(output)

print(translations)

true = []
for sentence in test_y:
  output = []
  for word in sentence:
    token = np.argmax(word)
    if token == 0:
      continue
    else:
      eng_word = index_to_word[token]
    output+= eng_word
  true.append(output)


results = pd.DataFrame({'true': true, 'trans': translations})



In [ ]:
results.to_csv('/content/results_string.csv', index=False)

In [54]:
import pandas as pd
results = pd.read_csv('/content/results.csv')


I have an older version of the results df downloaded and had to restart the runtime, so I will do some preprocessing again.

In [55]:
import ast
results['trans'] = results['trans'].apply(ast.literal_eval)
results['true'] = results['true'].apply(ast.literal_eval)

In [56]:
results['trans']

0       [i, wanted, to, sing, ]
1         [i, hope, you, die, ]
2         [you, are, young, , ]
3          [it, is, too, hot, ]
4          [we, will, wait, , ]
                 ...           
1981     [i, am, a, sculptor, ]
1982       [give, me, half, , ]
1983           [go, go, go, , ]
1984    [what, will, i, have, ]
1985      [open, a, window, , ]
Name: trans, Length: 1986, dtype: object

In [57]:
trans = results['trans']

translations = []
for row in trans:
  string = ''
  for token in row:
    if token == '':
      continue
    if token == 'ca': #in the preprocessing process I accidently replaced can't with ca not
      token = 'can'
    if token == 'wo': #in the preprocessing process I accidently replaced won't with wo not
      token = 'will'

    # we will get rid of some clearly equivlant translations, such as 'a' and 'the' being equivlant in russian

    if token == 'a':
      token = 'the'

    string+= token + ' '
  translations.append(string)

true = results['true']

trues = []
for row in true:
  string = ''
  for token in row:
    if token == '':
      continue
    if token == 'ca': #in the preprocessing process I accidently replaced can't with ca not
      token = 'can'
    if token == 'wo': #in the preprocessing process I accidently replaced won't with wo not
      token = 'will'

    # we will get rid of some clearly equivlant translations, such as 'a' and 'the' being equivlant in russian

    if token == 'a':
      token = 'the'

    string+= token + ' '
  trues.append(string)

trues = [sentence[:-1] for sentence in trues] # to get rid of extra spasce at the end
translations = [sentence[:-1] for sentence in translations]

In [58]:
trues[30:40]

['tom had the gun',
 'i am stubborn',
 'we can make it',
 'i got mad',
 'tom was angry',
 'i was not alone',
 'i read his book',
 'cover your legs',
 'i sold the book',
 'stop quibbling']

In [59]:
translations[30:40]

['tom had the gun',
 'i am contagious',
 'we will it it',
 'i was myself',
 'tom was angry',
 'i was not alone',
 'i read it book',
 'cover your feet',
 'i took the book',
 'toms famous']

In [ ]:
!pip install evaluate

In [61]:
import evaluate
bleu = evaluate.load('bleu')
predictions = translations
references = trues
results = bleu.compute(predictions=predictions, references=references,
          max_order = 2) # the sentences are very short so we will only use 2 n-grams
print(results)

{'bleu': 0.5969121417304739, 'precisions': [0.695322376738306, 0.526715799170889], 'brevity_penalty': 0.9863456581662835, 'length_ratio': 0.9864380358534685, 'translation_length': 6328, 'reference_length': 6415}
